In [1]:
import pandas as pd
import pyarrow.parquet as pq
from numpy import nan
import numpy as np

# Windmill Preprocessing
## Clean and extract windmill data

In [2]:
# static cols
cols = ['GSRN','Turbine_type','Parent_GSRN','BBR_municipal','Placement','UTM_x','UTM_y','Capacity_kw','Rotor_diameter','Navhub_height','Out_service']

In [3]:
windmills = pq.read_table('data/ITU_DATA/masterdatawind.parquet', columns=cols).to_pandas()

In [4]:
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service
0,000000000000000000,H,None,760,None,NaN,NaN,11.0,13.0,18.2,2014-10-01
1,570714700000000027,M,570714700000050459,101,LAND,720898.353,6171174.877,225.0,27.0,30.0,NaT
2,570714700000000027,M,570714700000050459,101,LAND,720898.353,6171174.877,225.0,27.0,30.0,NaT
3,570714700000000034,M,570714700000050459,101,LAND,720993.352,6171226.877,225.0,27.0,30.0,NaT
4,570714700000000034,M,570714700000050459,101,LAND,720993.352,6171226.877,225.0,27.0,30.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...
85145,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85146,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85147,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85148,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT


### Filter In_service from 2017 till now

In [5]:
windmills = windmills[windmills['Out_service'] > '2017-01-01'].append(windmills[windmills['Out_service'].isna()])

In [6]:
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service
413,570714700000001659,M,570714700000050015,360,HAV,635831.57,6093877.23,450.0,35.0,37.5,2017-02-20
416,570714700000001666,M,570714700000050015,360,HAV,636028.76,6093650.72,450.0,35.0,37.5,2017-02-20
418,570714700000001673,M,570714700000050015,360,HAV,636226.42,6093426.62,450.0,35.0,37.5,2017-02-20
421,570714700000001680,M,570714700000050015,360,HAV,636423.93,6093200.36,450.0,35.0,37.5,2017-02-20
425,570714700000001697,M,570714700000050015,360,HAV,636620.72,6092973.61,450.0,35.0,37.5,2017-02-20
...,...,...,...,...,...,...,...,...,...,...,...
85145,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85146,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85147,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT
85148,571313186000124581,H,None,751,None,NaN,NaN,20.0,8.0,13.0,NaT


### Remove duplicated data
Windmills have duplicated data by different reviewers.    

In [7]:
windmills.drop_duplicates(subset = "GSRN", keep = 'first', ignore_index = True, inplace = True)
windmills.shape

(9208, 11)

9208 Contains:    
- H: Household turbine : No UTM_xy locations 
- W: Single turbine : full data
- P: Turbine park : No UTM_xy locations
- M: Turbine in a park : No power record in settlement data


In [8]:
print('Household: ',windmills[windmills['Turbine_type'] == 'H'].shape)
print('Single: ',windmills[windmills['Turbine_type'] == 'W'].shape)
print('Park: ',windmills[windmills['Turbine_type'] == 'P'].shape)
print('In a Park: ',windmills[windmills['Turbine_type'] == 'M'].shape)

Household:  (2283, 11)
Single:  (3942, 11)
Park:  (441, 11)
In a Park:  (2542, 11)


### Fill Turbine park xy by calculate their CENTRIODs.

In [9]:
aggs = {'UTM_x':lambda x : x.mean(),
        'UTM_y':lambda x : x.mean()}
w_temp = windmills[windmills['Turbine_type'] == 'M'].groupby(['Parent_GSRN'], as_index = False).agg(aggs).rename(columns={'Parent_GSRN':'GSRN','UTM_x':'x_c','UTM_y':'y_c'})
w_temp

,GSRN,x_c,y_c
0,570714700000005640,642373.900000,6.139972e+06
1,570714700000012037,664423.733000,6.072868e+06
2,570714700000012051,664630.731000,6.072840e+06
3,570714700000050008,684100.400000,6.097951e+06
4,570714700000050015,636349.972727,6.093533e+06
...,...,...,...
501,570715000000089299,664492.611111,6.048460e+06
502,570715000000258107,636075.348135,6.275118e+06
503,570715000001613493,415507.469388,6.172906e+06
504,570715000001761682,464444.000000,6.283758e+06


#### (Option) Fill some missing turbine parks
506 != 441   


In [ ]:
f = windmills[windmills['Turbine_type'] == 'P']

In [ ]:
f[f['GSRN'].isin(w_temp['GSRN'])]

In [ ]:
f[~f['GSRN'].isin(w_temp['GSRN'])]

In [ ]:
w_temp[~w_temp['GSRN'].isin(f['GSRN'])]

Fill 136 in to windmills

In [ ]:
windmills.shape

#### Fill 

In [10]:
windmills = pd.merge(windmills, w_temp, on='GSRN', how='left')
# location
windmills[['UTM_x']] = windmills.apply(lambda x: x[['UTM_x']].fillna(value = x['x_c']), axis=1)[['UTM_x']]
windmills[['UTM_y']] = windmills.apply(lambda x: x[['UTM_y']].fillna(value = x['y_c']), axis=1)[['UTM_y']]
windmills = windmills.drop(columns=['x_c','y_c'])
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service
0,570714700000001659,M,570714700000050015,360,HAV,635831.57,6093877.23,450.0,35.0,37.5,2017-02-20
1,570714700000001666,M,570714700000050015,360,HAV,636028.76,6093650.72,450.0,35.0,37.5,2017-02-20
2,570714700000001673,M,570714700000050015,360,HAV,636226.42,6093426.62,450.0,35.0,37.5,2017-02-20
3,570714700000001680,M,570714700000050015,360,HAV,636423.93,6093200.36,450.0,35.0,37.5,2017-02-20
4,570714700000001697,M,570714700000050015,360,HAV,636620.72,6092973.61,450.0,35.0,37.5,2017-02-20
...,...,...,...,...,...,...,...,...,...,...,...
9203,571313174116645946,H,None,350,None,NaN,NaN,10.0,7.1,21.3,NaT
9204,571313179100075143,H,None,219,None,NaN,NaN,10.0,13.0,18.2,NaT
9205,571313179100118352,H,None,260,None,NaN,NaN,10.0,7.0,18.0,NaT
9206,571313179100992129,H,None,250,None,NaN,NaN,25.0,11.0,23.5,NaT


Cause a bug in ArcGIS file import, we need to add a 'z' at the end of GSRN and Parent_GSRN

In [11]:
windmills['GSRN'] = [x+'z' for x in windmills['GSRN']]
windmills['Parent_GSRN'].fillna('',inplace = True)
windmills['Parent_GSRN'] = [x+'z' for x in windmills['Parent_GSRN']]

In [12]:
windmills.to_csv('data/windmills_gis.csv', index=False)

### After ArcGIS    
Merge

In [14]:
cols = ['GSRN','grid']
windmills_single_grid= pd.read_excel('data/windmills_to_single-grid.xlsx')[cols]
windmills_single_grid['grid'].fillna(value=0, inplace = True)
windmills_single_grid['grid'] = windmills_single_grid['grid'].astype(int)
windmills_single_grid['grid'] = windmills_single_grid['grid'].astype(str)
windmills = pd.merge(windmills, windmills_single_grid, on='GSRN', how='left')
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service,grid
0,570714700000001659z,M,570714700000050015z,360,HAV,635831.57,6093877.23,450.0,35.0,37.5,2017-02-20,815
1,570714700000001666z,M,570714700000050015z,360,HAV,636028.76,6093650.72,450.0,35.0,37.5,2017-02-20,815
2,570714700000001673z,M,570714700000050015z,360,HAV,636226.42,6093426.62,450.0,35.0,37.5,2017-02-20,815
3,570714700000001680z,M,570714700000050015z,360,HAV,636423.93,6093200.36,450.0,35.0,37.5,2017-02-20,815
4,570714700000001697z,M,570714700000050015z,360,HAV,636620.72,6092973.61,450.0,35.0,37.5,2017-02-20,814
...,...,...,...,...,...,...,...,...,...,...,...,...
9203,571313174116645946z,H,z,350,None,NaN,NaN,10.0,7.1,21.3,NaT,0
9204,571313179100075143z,H,z,219,None,NaN,NaN,10.0,13.0,18.2,NaT,0
9205,571313179100118352z,H,z,260,None,NaN,NaN,10.0,7.0,18.0,NaT,0
9206,571313179100992129z,H,z,250,None,NaN,NaN,25.0,11.0,23.5,NaT,0


In [16]:
cols = ['GSRN','grid']
windmills_multi_grid = pd.read_excel('data/windmills_to_multi-grid.xlsx')[cols]
windmills_multi_grid['grid'].fillna(value=0, inplace = True)
windmills_multi_grid['grid'] = windmills_multi_grid['grid'].astype(int)
windmills_multi_grid['grid'] = windmills_multi_grid['grid'].astype(str)

In [17]:
def ab(df):
    return','.join(df.values)
    
windmills_multi_grid = windmills_multi_grid.groupby('GSRN')['grid'].apply(ab).reset_index().rename(columns={'grid':'grid_in_range'})
windmills = pd.merge(windmills, windmills_multi_grid, on='GSRN', how='left')
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service,grid,grid_in_range
0,570714700000001659z,M,570714700000050015z,360,HAV,635831.57,6093877.23,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85..."
1,570714700000001666z,M,570714700000050015z,360,HAV,636028.76,6093650.72,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85..."
2,570714700000001673z,M,570714700000050015z,360,HAV,636226.42,6093426.62,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85..."
3,570714700000001680z,M,570714700000050015z,360,HAV,636423.93,6093200.36,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85..."
4,570714700000001697z,M,570714700000050015z,360,HAV,636620.72,6092973.61,450.0,35.0,37.5,2017-02-20,814,"855,813,771,856,729,730,814,731,773,815,857,85..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9203,571313174116645946z,H,z,350,None,NaN,NaN,10.0,7.1,21.3,NaT,0,NaN
9204,571313179100075143z,H,z,219,None,NaN,NaN,10.0,13.0,18.2,NaT,0,NaN
9205,571313179100118352z,H,z,260,None,NaN,NaN,10.0,7.0,18.0,NaT,0,NaN
9206,571313179100992129z,H,z,250,None,NaN,NaN,25.0,11.0,23.5,NaT,0,NaN


In [18]:
cols = ['GSRN', 'Land_cover', 'Slope', 'Elevation']
windmills_gis_ext = pd.read_excel('data/windmills_lc_slope_ele.xlsx')[cols]
windmills_gis_ext['Land_cover'].fillna(value=-1, inplace = True)
windmills_gis_ext['Slope'].fillna(value=-1, inplace = True)
windmills_gis_ext['Elevation'].fillna(value=-1, inplace = True)

In [19]:
roughness_dic = {210:0.0, 220:0.004, 200:0.005, 202:0.005, 140:0.03, 150:0.05, 11:0.1, 14:0.1, 130:0.1, 180:11, 10:0.1, 20:0.3, 30:0.3, 160:0.5, 120:0.5, 170:0.6, 190:1.0, 40:1.5, 50:1.5, 60:1.5, 70:1.5, 90:1.5, 100:1.5, 110:1.5, -1:0.0}
windmills_gis_ext['Roughness'] = windmills_gis_ext['Land_cover'].apply(lambda x: roughness_dic[x])
windmills_gis_ext

,GSRN,Land_cover,Slope,Elevation,Roughness
0,570714700000004094z,11,0.000000,0.00,0.1
1,570714700000002885z,11,0.000000,0.00,0.1
2,570714700000002892z,11,0.000000,1.63,0.1
3,570714700000002908z,11,1.432096,0.91,0.1
4,570714700000002915z,11,1.012750,0.90,0.1
...,...,...,...,...,...
7405,570715000000003547z,11,2.263636,27.00,0.1
7406,570715000000002779z,11,0.000000,4.00,0.1
7407,570715000000002601z,130,0.000000,3.00,0.1
7408,570715000000056956z,110,0.000000,4.00,1.5


In [20]:
windmills_gis_ext = windmills_gis_ext.astype({'GSRN':'str'})
windmills = pd.merge(windmills, windmills_gis_ext, on='GSRN', how='left')
windmills

,GSRN,Turbine_type,Parent_GSRN,BBR_municipal,Placement,UTM_x,UTM_y,Capacity_kw,Rotor_diameter,Navhub_height,Out_service,grid,grid_in_range,Land_cover,Slope,Elevation,Roughness
0,570714700000001659z,M,570714700000050015z,360,HAV,635831.57,6093877.23,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85...",210.0,-1.0,-1.0,0.0
1,570714700000001666z,M,570714700000050015z,360,HAV,636028.76,6093650.72,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85...",210.0,-1.0,-1.0,0.0
2,570714700000001673z,M,570714700000050015z,360,HAV,636226.42,6093426.62,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85...",210.0,-1.0,-1.0,0.0
3,570714700000001680z,M,570714700000050015z,360,HAV,636423.93,6093200.36,450.0,35.0,37.5,2017-02-20,815,"855,813,771,856,729,730,814,731,773,815,857,85...",210.0,-1.0,-1.0,0.0
4,570714700000001697z,M,570714700000050015z,360,HAV,636620.72,6092973.61,450.0,35.0,37.5,2017-02-20,814,"855,813,771,856,729,730,814,731,773,815,857,85...",210.0,-1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9203,571313174116645946z,H,z,350,None,NaN,NaN,10.0,7.1,21.3,NaT,0,NaN,NaN,NaN,NaN,NaN
9204,571313179100075143z,H,z,219,None,NaN,NaN,10.0,13.0,18.2,NaT,0,NaN,NaN,NaN,NaN,NaN
9205,571313179100118352z,H,z,260,None,NaN,NaN,10.0,7.0,18.0,NaT,0,NaN,NaN,NaN,NaN,NaN
9206,571313179100992129z,H,z,250,None,NaN,NaN,25.0,11.0,23.5,NaT,0,NaN,NaN,NaN,NaN,NaN


### Save

In [21]:
windmills['GSRN'] = windmills['GSRN'].apply(lambda x: x[0:len(x)-1])
windmills['Parent_GSRN'] = windmills['Parent_GSRN'].apply(lambda x: x[0:len(x)-1])

In [22]:
windmills.to_csv('data/windmill_cleaned.csv',index=False)